In [93]:
!pip install transformers > None
!pip install datasets > None

In [90]:
!pip install evaluate > None

In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


Возьмем модель, которая уже умеет распознавать тональность текста (положительная/отрицательная) и попробуем докрутить ее на массиве отзывов на фильмы из первого задания.

Посмотрим как работает модель.

In [14]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'POSITIVE'

Загрузим наш датасет и разобьем его на тренировочную и тестовую выборки.

In [3]:
path = 'IMDB Dataset.csv'
data = pd.read_csv(path)
df = data.copy()
df['target'] = df['sentiment'].map({'negative': 0, 'positive': 1})
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['target'], test_size = 0.5, random_state = 123)


Функция токенизации:

In [15]:
def tokenize_function(examples):
    return tokenizer(examples["text"], return_tensors="pt", padding="max_length", truncation=True)

Преобразуем наш датасет в соответствующий объект для работы с инструментами HuggingFace

In [5]:
from datasets import load_dataset, DatasetDict, Dataset
my_dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':X_train}),
     'test':Dataset.from_dict({'label':y_test,'text':X_test})
     })

Токенизируем датасет:

In [16]:
tokenized_datasets = my_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Выделим тренировочный и тестовый датасеты

In [17]:
my_train_dataset = tokenized_datasets["train"]
my_eval_dataset = tokenized_datasets["test"]

In [18]:
my_eval_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

Выберем метрику (F1 мера) и добавим функцию для ее расчета



In [19]:
import evaluate

metric = evaluate.load("f1")

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [96]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.3 MB/s eta 0:00:00


Попробуем дообучить модель:

In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=my_train_dataset,
    eval_dataset=my_eval_dataset,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.271600,0.235914,0.923441
2,0.151700,0.288859,0.925452
3,0.060200,0.382778,0.933083


TrainOutput(global_step=9375, training_loss=0.1690055029296875, metrics={'train_runtime': 5140.4026, 'train_samples_per_second': 14.59, 'train_steps_per_second': 1.824, 'total_flos': 1.003043142623232e+16, 'train_loss': 0.1690055029296875, 'epoch': 3.0})

Судя по всему, модель начала становиться переобученной. Тем не менее целевая метрика растет. Почему?:)

Попробуем метод evaluate(). Хотел проверить внеслись ли в модель изменения. Ответ положительный.

In [26]:
trainer.evaluate()

{'eval_loss': 0.38277801871299744,
 'eval_f1': 0.9330825262146801,
 'eval_runtime': 436.1205,
 'eval_samples_per_second': 57.324,
 'eval_steps_per_second': 7.165,
 'epoch': 3.0}

Забыл попробовать оценить качество перед дообучением, поэтому сообразим "костыль":

In [29]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
trainer_no_learn = Trainer(
    model=model,
    args=training_args,
    train_dataset=my_train_dataset,
    eval_dataset=my_eval_dataset,
    compute_metrics=compute_metrics,
)

In [30]:
trainer_no_learn.evaluate()

{'eval_loss': 0.4178711473941803,
 'eval_f1': 0.8863031914893617,
 'eval_runtime': 430.329,
 'eval_samples_per_second': 58.095,
 'eval_steps_per_second': 7.262}

Получается, что за счет дообучения мы выиграли около 0.05 по целевой метрике, но есть вопрос с переобучением. Наверное следовало бы еще поиграться с моделями и их дообучением, но времени в рамках курса уже не остается.